In [1]:
import requests
import json
import sys
import torch
import io
import pandas as pd
from PIL import Image
import open_clip
import wandb
from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_parquet('~/data/bild/00000.parquet')

In [4]:
tc = nltk.classify.textcat.TextCat() 

idxs = []
for i in tqdm(range(df.shape[0])):
	if (tc.guess_language(df.iloc[i]['Text']) == 'eng') and (df.iloc[i]['Page_config']['img_count'] > 0):
		idxs.append(i)

	if i % 100 == 0:
		df_subset = df.iloc[idxs]
		df_subset.to_parquet('00000_subset.parquet')

A Jupyter Widget